In [ ]:
#!/usr/bin/env python3
"""sky_positional_matching.ipynb
James Gardner 2019

generates primitive feature vector catalogue for entire sky covered by
both surveys, elements are positional matches of one source from TGSS
and one source form NVSS with the SEPARATION_DIST of 2', note that this
is less than that of 10' for the smaller patch, this is done to improve runtime

standard execution, all cells top to bottom:
requires TGSSADR1_7sigma_catalog.tsv and CATALOG.FIT be present in cwd
produces sky_matches.csv, sky_catalogue.csv, hist_angle.pdf, and hist_alpha.pdf

alternative execution:
have the provided sky_matches.csv (found in source folder) present in cwd
comment out/delete the third (3rd) cell below that calls generate_matches (search for 'delete this')
execute all (remaining) cells from top to bottom
requires existing sky_matches.csv and well as
TGSSADR1_7sigma_catalog.tsv and CATALOG.FIT be present in cwd
modified sky_positional_matching.ipynb only saves
sky_catalogue.csv, hist_angle.pdf, and hist_alpha.pdf
"""

import numpy as np
from astropy.io import fits
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt

# separation limit, 2'
SEPARATION_DIST = 2*1/60

In [ ]:
def generate_matches():
    """generates sky_matches.csv, containing all pairs from tgss, nvss
    s.t. a basic estimate for their separation is closer than SEPARATION_DIST
    requires TGSSADR1_7sigma_catalog.tsv and CATALOG.FIT be present in cwd
    """
    # load ra,dec from tgss 
    tgss = np.genfromtxt(
        fname="TGSSADR1_7sigma_catalog.tsv",
        delimiter='\t',
        skip_header=1,
        usecols=(1,3))
    tgss = tgss[tgss[:,1].argsort()]    

    # load ra,dec from nvss
    with fits.open("CATALOG.FIT") as hdulist:
        data = hdulist[1].data
        nvss = np.column_stack((data['RA(2000)'],data['DEC(2000)']))
        # uncomment lines to examine NVSS catalogue
        #hdulist.info()
        #hdr = hdulist[0].header
        #print(repr(hdr))
        #cols = hdulist[1].columns
        #cols.info()    
    nvss = nvss[nvss[:,1].argsort()]

    print('survey lengths',np.shape(tgss),np.shape(nvss))

    # constructing 0.1 deg lookup bins of nvss to speed up
    # for loop of the cross of the two surveys
    nvss_dec_min = round(nvss[:,1].min(),1)
    nvss_dec_max = round(nvss[:,1].max(),1)

    # chunkis is index of where bin starts
    bin_size = 0.1
    mark = nvss_dec_min - bin_size
    chunkis = []
    count = 0
    # finding the first element in each bin
    for i,dec in enumerate(nvss[:,1]):    
        if mark < dec < mark + bin_size:
            chunkis.append(i)
            mark += bin_size

    bins = [x/10 for x in range(int((nvss_dec_min-bin_size)*10),
                                int((nvss_dec_max+bin_size+0.01)*10))]
    cos_dec = np.array([np.cos(dec*np.pi/180) for dec in bins])

    # useful lines to check bin creation successful
    #print(chunkis[:5],'...',chunkis[-5:])
    #print(bins[:5],'...',bins[-5:])

    matches = []
    bar = tqdm(total=len(tgss))

    # warning: this can take at least 30 minutes to complete
    for i1,p1 in enumerate(tgss):
        # using the sorting of nvss
        if p1[1] < nvss_dec_min - 0.1:
            bar.update(1)
            continue
        elif p1[1] > nvss_dec_max + 0.1:
            break

        which_bin = bins.index(np.floor(p1[1]*10)/10)
        # look both ways for which bin: -1, 0, +1
        nslice = chunkis[which_bin-1],chunkis[which_bin+2]

        for i2,p2 in enumerate(nvss[nslice[0]:nslice[1]]):
            # careful with azimuth near poles
            if (abs((p1[0]-p2[0])*cos_dec[which_bin]) < SEPARATION_DIST
                    and abs(p1[1]-p2[1]) < SEPARATION_DIST):
                matches.append((p1,p2))

        bar.postfix = 'matches = {}'.format(len(matches))
        bar.update(1)

    # only crude matches, would have been nice to save the names with these
    matches = np.array([(m[0][0],m[0][1],m[1][0],m[1][1]) for m in matches])
    np.savetxt('sky_matches.csv', matches, delimiter=',')

In [ ]:
# comment out/delete this (and only this) cell for alternative bypass
# must have existing sky_matches.csv present in cwd for bypass along with surveys
generate_matches()

In [ ]:
def geodist(p1,p2):
    """given two points close on the unit sphere,
    return their geodesic distance,
    input points must be in radians!"""
    ra1,dec1,ra2,dec2 = p1[0],p1[1],p2[0],p2[1]
    # see https://en.wikipedia.org/wiki/Great-circle_distance#Formulae
    decdiff = (dec1-dec2)/2
    radiff  = (ra1-ra2)/2
    better_circle = 2*np.arcsin(np.sqrt(np.sin(decdiff)**2
                    + np.cos(dec1)*np.cos(dec2) * np.sin(radiff)**2))
    r = 1
    return better_circle*r

def degdist(p1,p2):
    """calls geodist on argument points in degrees,
    after converting from degrees to radians and back"""
    return 180/np.pi*geodist([x*np.pi/180 for x in p1],
                             [x*np.pi/180 for x in p2])

In [ ]:
# here's where the alternative bypass will pickup matches
matches = np.loadtxt('sky_matches.csv',delimiter=',')
matches = np.reshape(matches,(len(matches), 2, 2))
# print('initial matches:',matches.shape)

# filter with the proper geodesic distance metric
# and begin constructing the catalogue
# rows are to be: name_TGSS, ra_TGSS, dec_TGSS, name_NVSS, ra_NVSS, dec_NVSS,
# separation, non_unique_flag, peak_flux_TGSS, peak_flux_NVSS, alpha
catalogue = []
for m in tqdm(matches):
    # remember to maintain the established order of: tgss, nvss
    p1,p2 = m
    d = degdist(p1,p2)
    if d < SEPARATION_DIST:
        catalogue.append(('',p1[0],p1[1],\
                          '',p2[0],p2[1],\
                          d,0,0,0,0))
catalogue = np.array(catalogue,dtype=object)
# nice to know total number of matches
print('catalogue:',catalogue.shape)

In [ ]:
# flag all non-one-to-one matches between TGSS and NVSS
for i,m in enumerate(tqdm(catalogue)):
    p1ra,p1dec,p2ra,p2dec = m[1],m[2],m[4],m[5]
    # abusing sorting, check only those nearby for non-uniqueness
    # nearby value is arbitrary, in testing nothing greater than 10 was needed
    nearby = 10
    if nearby < i < len(catalogue)-nearby:
        rest_wo = np.concatenate((catalogue[i-nearby:i],catalogue[i+1:i+1+nearby]))
    elif i < nearby:
        rest_wo = np.concatenate((catalogue[:i],catalogue[i+1:i+1+nearby]))
    elif len(catalogue)-nearby < i:
        rest_wo = np.concatenate((catalogue[i-nearby:i],catalogue[i+1:]))

    tgss_wo = rest_wo[:,(1,2)]
    nvss_wo = rest_wo[:,(4,5)]
    
    if (np.any((tgss_wo[:]==[p1ra,p1dec]).all(1)) or
        np.any((nvss_wo[:]==[p2ra,p2dec]).all(1))):
        catalogue[i][7] = 1

In [ ]:
# reload surveys to recover names, fluxes, and calculate alpha
tgss_labels = np.genfromtxt('TGSSADR1_7sigma_catalog.tsv',
                            delimiter='\t', skip_header=1, usecols=0, dtype=str)
ftgss = np.genfromtxt('TGSSADR1_7sigma_catalog.tsv',
                      delimiter='\t', skip_header=1, usecols=(1,3,7))
# unfortunate choice of variable name here, labels in the naming, not the ML sense
tgss_labels = tgss_labels[ftgss[:,1].argsort()]
ftgss = ftgss[ftgss[:,1].argsort()]

In [ ]:
with fits.open("CATALOG.FIT") as hdulist:
    data = hdulist[1].data
    # jansky?!/beam, is confusing units
    # note that I don't correct for beamsize, this should have been done
    fnvss = np.column_stack((data['RA(2000)'],data['DEC(2000)'],data['PEAK INT']))
fnvss = fnvss[fnvss[:,1].argsort()]

In [ ]:
def deci_deg_to_deg_min_sec(deci_deg):
    """converts degree measurement into degrees-minutes-seconds
    this is a clean solution, credit goes to:
    (https://stackoverflow.com/questions/2579535/
    convert-dd-decimal-degrees-to-dms-degrees-minutes-seconds-in-python)
    """
    is_positive = (deci_deg >= 0)
    deci_deg = abs(deci_deg)
    # divmod returns quotient and remainder
    minutes,seconds = divmod(deci_deg*3600,60)
    degrees,minutes = divmod(minutes,60)
    degrees = degrees if is_positive else -degrees
    return (degrees,minutes,seconds)

def deci_deg_to_hr_min_sec(deci_deg):
    """converts degree measurement into hours-minutes-seconds
    assumes that deci_deg is postitive"""
    deci_hours = deci_deg/15.
    schminutes,schmeconds = divmod(deci_hours*3600,60)
    hours,schminutes = divmod(schminutes,60)   
    return (hours,schminutes,schmeconds)

In [ ]:
def iau_designation(ra,dec):
    """generate NVSS names as per:
    https://heasarc.gsfc.nasa.gov/W3Browse/all/nvss.html
    There are four cases where there are pairs of sources which are
    so close together that their names would be identical according
    to this schema (see below), and the HEASARC has added suffixes
    of 'a' (for the source with smaller RA) and 'b' (for the source
    with the larger RA) in such cases in order to differentate them.
    It was easier just to hard-code this in,
    should really check if designation alreadys exists and compare
    """
    hr,schmin,schmec = deci_deg_to_hr_min_sec(ra)
    rhh = str(int(hr)).zfill(2)
    rmm = str(int(schmin)).zfill(2)
    rss = str(int(schmec - schmec%1)).zfill(2)

    deg,minu,sec = deci_deg_to_deg_min_sec(dec)
    sgn = '+' if deg>=0 else '-'
    ddd = str(int(abs(deg))).zfill(2)
    dmm = str(int(minu)).zfill(2)
    dss = str(int(sec - sec%1)).zfill(2)

    designation = ''.join(('NVSS J',rhh,rmm,rss,sgn,ddd,dmm,dss))
    
    close_pairs = {'NVSS J093731-102001':144.382,
                   'NVSS J133156-121336':202.987,
                   'NVSS J160612+000027':241.553,
                   'NVSS J215552+380029':328.968}
    if designation in close_pairs:
        if ra < close_pairs[designation]:
            designation = ''.join((designation,'a'))
        else:
            designation = ''.join((designation,'b'))         

    return designation

In [ ]:
# again, an unfortunate variable name, these are names of sources, not ML labels
nvss_labels = np.array([iau_designation(p[0],p[1]) for p in tqdm(fnvss)])

In [ ]:
# tgss in mJy/beam, beams in ''
# here I commented out the beam correction, I cannot justify why
# yet the spectral alpha comparison to the reference paper seems to hold
tgssbeam = 25
ftgss[:,2] = ftgss[:,2]*1e-3#*tgssbeam
# nvss in Jy/beam
nvssbeam = 45
fnvss[:,2] = fnvss[:,2]#*nvssbeam

In [ ]:
# add labels, flux densities, and spectral index to catalogue
# again, rows are: name_TGSS, ra_TGSS, dec_TGSS, name_NVSS, ra_NVSS, dec_NVSS,
# separation, non_unique_flag, peak_flux_TGSS, peak_flux_NVSS, alpha
FREQ_NVSS, FREQ_TGSS = 1.4e9,150e6

for i,m in enumerate(tqdm(catalogue)):
    tdec,ndec = m[2],m[5]
    # searchsorted gives index value if equal to or next if not
    ti = np.searchsorted(ftgss[:,1],tdec)
    ni = np.searchsorted(fnvss[:,1],ndec)
    t_name,n_name = tgss_labels[ti],nvss_labels[ni]
    s_tgss,s_nvss = ftgss[ti][2],fnvss[ni][2]
    alpha = np.log(s_tgss/s_nvss)/np.log(FREQ_NVSS/FREQ_TGSS)
    
    catalogue[i][0] = t_name
    catalogue[i][3] = n_name
    catalogue[i][8] = s_tgss
    catalogue[i][9] = s_nvss
    catalogue[i][10] = alpha

catalogue_fmt = ('%s','%1.5f','%1.5f','%s','%1.14f','%1.14f',
                 '%1.18f','%i','%1.5f','%1.5f','%1.5f')
np.savetxt('sky_catalogue.csv', catalogue, delimiter=',',fmt = catalogue_fmt)

In [ ]:
seps = [3600*val for val in catalogue[:,6]]

# create histogram of angular separation of matches
plt.figure(figsize=(14,7))
plt.rcParams.update({'font.size': 18})
plt.hist(seps, bins=100,color = "darkmagenta", ec="orchid")
plt.xlabel("angular separation, '' (arcsec)")
plt.ylabel('counts')
plt.title('distribution of angular separation of matches')
plt.savefig('hist_angle.pdf',bbox_inches='tight')

In [ ]:
# create histogram of alpha value of matches, to compare to the reference paper
plt.rcParams.update({'font.size': 18})
alpha = catalogue[:,10]
s_tgss = catalogue[:,8]*1e3
# thresholds as in paper
allalpha = alpha,alpha[s_tgss>50],alpha[s_tgss>100],alpha[s_tgss>150],alpha[s_tgss>200]

fig,axis = plt.subplots(figsize=(14,7))
plt.hist(allalpha, bins=200, histtype='step', stacked=False, fill=False,\
         label=['no cut','S_tgss>50mJy','S_tgss>100mJy','S_tgss>150mJy','S_tgss>200mJy'],\
         color=['black','limegreen','orange','magenta','darkblue'])
plt.legend(loc='upper right')
handles, labels = axis.get_legend_handles_labels()
plt.legend(reversed(handles), reversed(labels))
plt.xlim(0,2.5)
plt.xlabel("observed spectral index")
plt.ylabel('counts')
plt.title('distribution of observed spectral index of matches')
plt.savefig('hist_alpha.pdf',bbox_inches='tight')